<center><img src="https://images.twinkl.co.uk/tw1n/image/private/t_630/image_repo/b8/06/t-lf-242-pupil-voice-learning-child-led-learning-peer-assessment-cards_ver_1.jpg" height="100"></center>

# Detección de incoherencias en evaluación por pares

Profesor: Juan Ramón Rico (<juanramonrico@ua.es>)

## Descripción
---
No cabe duda que las redes neuronales han avanzado en tareas donde se usa texto y valores numéricos. Concretametne en las encuestas o en la evaluación por pares es habitual encontrar valores numéricos (libres o sujetos a una escala - Likert) y comentarios de los evaluadores respecto a las secciones evaluadas en forma de texto (feedback).

- Artículo sobre evaluación por pares donde se han usado redes recurrentes entre otras metodologías para detectar incongruencias en las respuestas <https://www.sciencedirect.com/science/article/pii/S0360131519301629?dgcid=author> ha servido como base para esta actividad.
---


# Introducción

El uso de la evaluación por pares para actividades abiertas tiene ventajas tanto para los profesores como para los estudiantes. Los profesores pueden reducir la carga de trabajo del proceso de corrección y los estudiantes logran una mejor comprensión de la materia al evaluar las actividades de sus compañeros. Para facilitar el proceso, es aconsejable proporcionar a los estudiantes una rúbrica sobre la cual realizar la evaluación de sus compañeros; sin embargo, limitarse a proporcionar sólo puntuaciones numéricas es perjudicial, ya que impide proporcionar una retroalimentación valiosa a otros compañeros. Dado que esta evaluación produce dos modalidades de la misma evaluación, a saber, la puntuación numérica y la retroalimentación textual, es posible aplicar técnicas automáticas para detectar inconsistencias en la evaluación, minimizando así la carga de trabajo de los profesores para supervisar todo el proceso.

Esta actividad estará enfocada en solo una parte de la detección de incongruencias que será la predicción de calificación de una sección usando únicamente información textual.

# Conjunto de datos

Los datos que vamos a utilizar para esta se pueden descargar en un archivo tipo CSV donde figura:

- `activity`: es la actividad desarrollada, en este caso 1 o 2. Son actividades independientes y los más lógico es entrenar modelos (redes neuronales) independientes para cada una;
-  `year`:  año de comienzo del curso académico estudiado;
- `group`: con valores de 1 si es de mañana o 2 si es de tarde;
- `evaluator`: identificador interno del evaluador para una actividad concreta (la actividad 1 o la 2, de forma excluyente); 
- `work` : es el identificador del trabajo. La entrega se realizaba mediante una URL por lo que hay ocasiones en la que es privada o no accesible y no se ha podido evaluar.
- `secction`: número de sección que se evalua dentro de cada actividad, o 'grade1', 'grade2' cuando se trata de la nota final del trabajo evaluado.
- `value`: valor numérico comprendido entre 0 y 3. Siendo 0 no realizado o realizado incorrectamente y 3 realizado correctamente. Este número puede tener decimales debido a que corresponde al promedio de los valores de la sección. 
- `feedback`: texto libre correspondiente a las recomendaciones que el evaluador realiza en cada sección. Puede estar en blanco lo que indica que no se realizan comentarios y corresponde a que todo está correcto.

Para esta actividad necesitaremos `activity`, `value` y `feeback`.

In [121]:
import numpy as np
import pandas as pd

data = pd.read_csv('https://www.dlsi.ua.es/~juanra/UA/dataset/dcadep/dcadep_melt_grades.csv.gz', sep='\t', decimal=',')
data.fillna('', inplace=True) # Reemplazar los valores en blanco por cadena vacías
data = data.sample(frac=1, random_state=123) # Barajar los ejemplos de entrada (filas de la tabla)
display(data.head())
data.dtypes

activity  year  group  evaluator  work section  value  \
23557         2  2015      2        103    59       7    2.0   
5292          1  2016      2        304   141       3    3.0   
22491         2  2016      2        333   137       6    3.0   
9655          1  2015      1         12    13  grade1    9.3   
25778         2  2016      1        204    88  grade2    9.4   

                                    feedback  
23557  Faltan datos para completar la página  
5292     Poco original y trabajo muy básico.  
22491                               Correcto  
9655                                          
25778

activity       int64
year           int64
group          int64
evaluator      int64
work           int64
section       object
value        float64
feedback      object
dtype: object

El atributo `section` contiene los identificadores de las diferentes secciones, así como las calificación final (`grade1` y `grade2` cuyos valores están entre 0 y 10) según la actividad. Esta actividad se tiene que evaluar por separado para la actividad 1 o 2 usando únicamente las secciones numéricas (1,2,3,4,5,6 y 7) cuyos valores oscilan entre 0 y 3.

In [122]:
np.sort(data['section'].unique())

array(['1', '2', '3', '4', '5', '6', '7', 'grade1', 'grade2'],
      dtype=object)

# Visualizar las valoraciones

In [123]:
data[data['section'].isin(['1','2','3','4','5','6','7'])][['activity','value']].groupby('activity').hist()

activity
1    [[AxesSubplot(0.125,0.125;0.336957x0.755), Axe...
2    [[AxesSubplot(0.125,0.125;0.336957x0.755), Axe...
dtype: object

Podemos ver como la mayoría de valores son cercanos a 3. No obstante tenemos que predecir cuando este valor va disminuyendo atendiendo a las palabras usadas en el contexto restringido de cada actividad.

# Preprocesar el texto

Es necesario preprocesar el texto para descartar símbolos de puntuación, valorar igualmente a palabras en mayúscula y minúscula y extraeer la raiz de las palabras (lemas) para procesarlas como iguales.

Para ello usaremos bibliotecas de procesamiento de lenguaje natural (PLN).

In [124]:
import nltk
nltk.download('stopwords')
  
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
stemmer = SnowballStemmer("spanish",ignore_stopwords=True)

preprocessed_feedback = []
for i in data.feedback:
  tokens = [stemmer.stem(word) for word in tokenizer.tokenize(i.lower())]
  preprocessed_feedback.append(np.array(' '.join(tokens)))

data['feedback prep'] = preprocessed_feedback
data['feedback prep'] = data['feedback prep'].astype('str')
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


activity  year  group  evaluator  work section  value  \
23557         2  2015      2        103    59       7    2.0   
5292          1  2016      2        304   141       3    3.0   
22491         2  2016      2        333   137       6    3.0   
9655          1  2015      1         12    13  grade1    9.3   
25778         2  2016      1        204    88  grade2    9.4   

                                    feedback                     feedback prep  
23557  Faltan datos para completar la página    falt dat para complet la pagin  
5292     Poco original y trabajo muy básico.  poco original y trabaj muy basic  
22491                               Correcto                           correct  
9655                                                                            
25778

In [125]:
data.dtypes

activity           int64
year               int64
group              int64
evaluator          int64
work               int64
section           object
value            float64
feedback          object
feedback prep     object
dtype: object

# Convertir datos de entrenamiento a la forma correcta

En este caso los datos de tipo texto hay transformarlos en secuencias de número que recibirá la red neuronal. 

- En el siguiente enlace <https://www.programcreek.com/python/example/106871/keras.preprocessing.text.Tokenizer> hay varios ejemplos de como convertir el texto que ya tenemos preprocesado a secuencias de números;
- De la secuencia de números hay que aplicar `pad_sequences` como truco para igualar la longitud de todas las secuencias de entreada. Nos facilita la tarea de entrenar un red recurrente con `Keras`.


In [126]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras_preprocessing.sequence import pad_sequences
# Seleccionar la actividad 1 y sus secciones, ya que se evaluan por separado. La actividad 2 se seleccionaría por separado
new_data = data[(data['activity']==1) & data['section'].isin(['1','2','3','4','5','6','7'])]

tokenizer = Tokenizer()
textitos = new_data['feedback prep']
tokenizer.fit_on_texts(textitos)
#print(tokenizer.word_counts)
sequences = []
for seq in tokenizer.texts_to_sequences_generator(textitos):
    sequences.append(seq)
# print(sequences)

# tokenizer.fit_on_sequences(sequences)
# matrix = tokenizer.sequences_to_matrix(sequences)
# print(matrix[0][0])

# Modes:
# 1) binary : The default value that tells us about the presence of each word in a document.
# 2) count : As the name suggests, the count for each word in the document is known.
# 3) tfidf : The TF-IDF score for each word in the document.
# TF-IDF or Term Frequency – Inverse Document Frequency, works by checking the relevance of a word in a given text corpus.
# In this mode, a proportional score is given to words on the basis of the number of times they occur in the text corpus. 
# In this way, this model can determine which words are worthy and which aren’t.
# 4) freq : The frequency tells us about ratio of words in each document.
#matrix = tokenizer.texts_to_matrix(textitos, "tfidf")

X = pad_sequences(sequences) # Valores de new_data['feedback prep'] con el padding aplicado
y = new_data['value'].values
totalPalabras = sum([x for _,x in tokenizer.word_counts.items()])
vocabSize = len(tokenizer.word_counts.items()) +1
lenSeq = len(X[0])
print(vocabSize)
print(X[2])
#print(vocabSize)
#print(tokenizer.word_counts.items())
#print(totalPalabras)


1679
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0  29  14  22  21  45  87 386   8   3  60
  43  10  27   2  21  12]


# Creando la red neuronal

Se han preprocesado los datos y los hemos convertido al formato correcto. Ahora tenemos que diseñar nuestra red para entrenarla y realizar las predicciones.

En el caso de codificar la entrada como un conjunto de palabras (Bag of Words) tendremos un vector de entrada con todas las palabras posibles del vocabulario. Cada posición del vector representa si está presente la palabra o no, el número de veces que se repite, o el valor TF-IDF correspondiente. La dimensión incial de la red cuando entrena será de `(batch_size, vocabulary_len)`.

Otro caso diferente corresponde a una codificación del la entrada atendiendo a una secuencia. Cada valor (entero) del vector de entrada representa a una palabra y la entrada cumple con estas tres dimensiones `(batch_size, time_steps, seq_len)` que necesita una red recurrente.

In [127]:
!pip install -q livelossplot

In [128]:
from tensorflow import keras
from tensorflow.keras.models import Sequential  
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Flatten, SimpleRNN, Bidirectional, Concatenate, Input, GlobalAveragePooling1D, Attention
from livelossplot import PlotLossesKeras


input_shape = 0# El que corresponda

activation = "relu"
batch_size = 128
MBEDDING_DIM = 128
max_len = lenSeq

# def rnn_model():

#   model = Sequential()

#   model.add(Embedding(input_dim=vocabSize, output_dim = batch_size))

#   model.add(Dense(64, activation=activation))

#   model.add(Dense(1))
#   return model
  
# model = keras.Sequential([
#     keras.layers.Embedding((vocabSize), 64, mask_zero=True),
#     keras.layers.Bidirectional(keras.layers.LSTM(64,  return_sequences=True)),
#     keras.layers.Bidirectional(keras.layers.LSTM(32)),
#     keras.layers.Dense(64, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(1)
# ])

# model = keras.Sequential()
# model.add(keras.layers.Dense(16, activation='sigmoid'))
# model.add(keras.layers.Dense(1))

# model = keras.Sequential([
#     keras.layers.Dense(256,input_dim=vocabSize, activation=activation),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(128, activation=activation),
#     keras.layers.Dense(64, activation=activation),
#     keras.layers.Dense(32, activation=activation),
#     keras.layers.Dense(16, activation=activation),
#     keras.layers.Dense(8, activation=activation),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(1)
# ])


# hay que rellenar la tipología de la red orientada a predecir un valor entre 0 y 3 (modelos de regresión)
#model = rnn_model()
# model = keras.Sequential([
#     keras.layers.Embedding(
#         input_dim=vocabSize,
#         output_dim=batch_size,
#         # Use masking to handle the variable sequence lengths
#         mask_zero=True),
#     keras.layers.Bidirectional(keras.layers.LSTM(64)),
#     keras.layers.Dense(64, activation='relu'),
#     keras.layers.Dense(1)
# ])

# model = keras.Sequential([
#     keras.layers.Dense(1)
# ])

# model = Sequential()
# embedding_layer = Embedding(input_dim=vocabSize,output_dim=4,input_length=15)
# model.add(embedding_layer)
# model.add(Flatten())
# model.add(Dense(1))

# model = Sequential()
# model.add(
#     layers.Embedding(input_dim=vocabSize,
#                      output_dim=128,
#                      input_length=lenSeq,
#                      mask_zero=True))
# model.add(
#     layers.SimpleRNN(16,
#                      activation="relu",
#                      name="RNN_layer",
#                      dropout=0.2, ## dropout for input character
#                      recurrent_dropout=0.2))  ## dropout for previous state
# model.add(layers.Dense(16, activation="relu", name="dense_layer"))
# model.add(layers.Dense(1, name="output"))


# model = Sequential()
# model.add(
#     Embedding(input_dim=vocabSize,
#               output_dim=128,
#               input_length=lenSeq,
#               mask_zero=True))
# model.add(Bidirectional(LSTM(16, dropout=0.5, recurrent_dropout=0.5)))
# model.add(Dense(1))
# 20 epochs
# Accuracy:  0.6221052631578947
# mae:  0.5747368421052632
# rsme:  0.9899494936611666
# [[ 49  16  11   9]
#  [  4  32  34  43]
#  [  4  75 136 151]
#  [  2 294  75 965]]
# Si subimos los dropout de 0.2 a 0.3 ==> sin efecto?
# Accuracy:  0.6231578947368421
# mae:  0.5805263157894737
# rsme:  1.0007891622956462
# [[ 50  13  11  11]
#  [  3  32  28  50]
#  [  4  72 135 155]
#  [  2 296  71 967]]
# Subiendo ambos dropout a 0.5 --> no importa
# Accuracy:  0.628421052631579
# mae:  0.5721052631578948
# rsme:  0.992869313497104
# [[ 50  15  10  10]
#  [  2  32  30  49]
#  [  6  65 135 160]
#  [  2 292  65 977]]


# ## Functional API
# inputs = Input(shape=(max_len, ))
# x = Embedding(input_dim=vocabSize,
#               output_dim=128,
#               input_length=max_len,
#               mask_zero=True)(inputs)
# _, x_last_h, x_c = LSTM(16,
#                         dropout=0.2,
#                         recurrent_dropout=0.2,
#                         return_sequences=False,
#                         return_state=True)(x)
# ## LSTM Parameters:
# #     `return_seqeunces=True`: return the hidden states for each time step
# #     `return_state=True`: return the cell state of the last time step
# #     When both are set True, the return values of LSTM are:
# #     (1) the hidden states of all time steps (when `return_sequences=True`) or the hidden state of the last time step
# #     (2) the hidden state of the last time step
# #     (3) the cell state of the last time step

# x = Concatenate(axis=1)([x_last_h, x_c])
# outputs = Dense(1)(x)
# model = Model(inputs=inputs, outputs=outputs)

# Accuracy:  0.6173684210526316
# mae:  0.5810526315789474
# rsme:  0.993134325997085
# [[ 49  13  17   6]
#  [  7  33  30  43]
#  [  7  71 142 146]
#  [  2 294  91 949]]



inputs = Input(shape=(lenSeq, ))
x = Embedding(input_dim=vocab_size,
              output_dim=128,
              input_length=lenSeq,
              mask_zero=True)(inputs)
x_all_hs, x_last_h, x_last_c = LSTM(32,
                                    dropout=0.2,
                                    recurrent_dropout=0.2,
                                    return_sequences=True,
                                    return_state=True)(x)
## LSTM Parameters:
#     `return_seqeunces=True`: return the hidden states for each time step
#     `return_state=True`: return the cell state of the last time step
#     When both are set True, the return values of LSTM are:
#     (1) the hidden states of all time steps (when `return_sequences=True`) or the hidden state of the last time step
#     (2) the hidden state of the last time step
#     (3) the cell state of the last time step

## Self Attention
atten_out = Attention()([x_all_hs, x_all_hs])  # query and key
atten_out_average = GlobalAveragePooling1D()(atten_out)

x_last_h_plus_atten = Concatenate()([x_last_h, atten_out_average])
x = Dense(16, activation="relu")(x_last_h_plus_atten)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs=inputs, outputs=outputs)



model.compile(optimizer = 'adam', loss = 'mean_absolute_error') 

# Entrenamiento del modelo

Vamos a dividir el conjunto en entrenamiento y test para comprobar nuestro modelo. Para ello tenemos que elegir un tipo de actividad (1 o 2), sus valores y feedback (preprocesado).

In [129]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.utils.multiclass import type_of_target

current_activity = 1 # 1 or 2

new_data = data[data.activity==current_activity]

# skf = StratifiedKFold(n_splits = 10, random_state = 890, shuffle= True)
# print(type_of_target(y))
# print(type_of_target(X))
# print(X)
# print(y)

# for train_index, test_index in skf.split(X,y):
#   #print("TRAIN:", train_index, "TEST:", test_index)
#   X_train, X_test = X[train_index], X[test_index]
#   y_train, y_test = y[train_index], y[test_index]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=890)
y_test=np.rint(y_test)
y_train=np.rint(y_train)

n = int(len(X)*0.9)

# X_train, X_test = X[:n], X[n:]
# y_train, y_test = y[:n], y[n:]
print(y_train[1])
print(X_train[1])

print(len(X_train))

3.0
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   4  81   7 456]
7600


In [130]:
# Para verificar que el modelo entrena correctamente crearemos un conjunto de validación del 10% de los datos con el parámetro ()
patience = 25
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
history=model.fit(X_train, y_train, epochs = 40, validation_split=0.1, batch_size = batch_size, callbacks=[callback, PlotLossesKeras()])

Loss
	training         	 (min:    0.706, max:    1.773, cur:    0.706)
	validation       	 (min:    0.779, max:    1.211, cur:    0.779)
54/54 [==============================] - 19s 351ms/step - loss: 0.7065 - val_loss: 0.7795


# Predicciones con el test

Las predicciones  se realizan entre 0 y 3 que son los valores mínimos y máximo establecidos para cada valorar cada ítem.

In [131]:
#y_pred = model.predict(X_test)
y_pred = np.clip(model.predict(X_test), 0, 3)

60/60 [==============================] - 2s 26ms/step


# Visualización de resultados

Tenemos que crear un DataFrame con el texto del feedback, valores reales, los predichos y su diferencia para apreciar las diferencias.

In [132]:
cero, uno, dos, tres = 0,0,0,0
y2 = []
x2 = []
for i in range(len(y_train)):
#   print(y_test[i])
    if y_train[i] == 0:
      cero = cero + 1
    if y_train[i] == 1:
      uno = uno + 1
    if y_train[i] == 2:
      dos = dos + 1
    if y_train[i] == 3:
      tres = tres + 1

print(cero, uno, dos, tres)

281 501 1467 5351


In [133]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_absolute_error, mean_squared_error
import math  
maeFunc = keras.losses.MeanAbsoluteError()
#df = pd.DataFrame(data, columns= np.array(data.columns))
#model.evaluate(X_test, y_test)
y_pred=np.rint(y_pred)
print("Accuracy: ", accuracy_score(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
rsme = mean_squared_error(y_test, y_pred, squared = False)

print('mae: ', mae)
print('rsme: ', rsme)
# Si ambos valores se pareces es que esta acertando bastante. 
# Si rsme es mucho mas grande (el doble o mas) no esta acertando bien

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

Accuracy:  0.5921052631578947
mae:  0.781578947368421
rsme:  1.3554956366778284
[[ 42  18  16   9]
 [ 13  12  42  46]
 [ 53  18 144 151]
 [284   9 116 927]]
              precision    recall  f1-score   support

         0.0       0.11      0.49      0.18        85
         1.0       0.21      0.11      0.14       113
         2.0       0.45      0.39      0.42       366
         3.0       0.82      0.69      0.75      1336

    accuracy                           0.59      1900
   macro avg       0.40      0.42      0.37      1900
weighted avg       0.68      0.59      0.63      1900

